## Common part

In [ ]:
!pip install -q gwpy

     |████████████████████████████████| 1.4 MB 12.9 MB/s 
     |████████████████████████████████| 51 kB 6.6 MB/s 
     |████████████████████████████████| 45 kB 3.1 MB/s 
     |████████████████████████████████| 11.2 MB 45.6 MB/s 
     |████████████████████████████████| 4.0 MB 47.6 MB/s 
     |████████████████████████████████| 957 kB 34.4 MB/s 


In [ ]:
# == Mount google colab folder
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os

INPUT_VIDEO = "/content/drive/MyDrive/kaggle/datasets/dfl-bundesliga-data-shootout/clips/08fd33_0.mp4"
# INPUT_VIDEO = "/content/drive/MyDrive/kaggle/datasets/dfl-bundesliga-data-shootout/train/1606b0e6_0.mp4"
# INPUT_VIDEO = "/content/drive/MyDrive/1606b0e6_0.mp4"
CONFIG_FILE = "/content/drive/MyDrive/kaggle/config.yaml"
TEMP_DIR = "/content/tmp_video"
OUTPUT_DIR = "/content/clip_video"
input_video_fn = os.path.basename(INPUT_VIDEO)
input_video_name = input_video_fn.split('.')[0]

if not os.path.exists(TEMP_DIR):
    os.makedirs(TEMP_DIR)
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

## Use bytetrack for given video

In [ ]:
%%capture
# == Download the repo content and install dependencies ==
!git clone https://github.com/ifzhang/ByteTrack.git
%cd /content/ByteTrack/
%mkdir pretrained
%cd pretrained

# == Download pretrained X model weights ==
!gdown --id "1P4mY0Yyd3PPTybgZkjMYhFri88nTmJX5"
!gdown --id "11Zb0NN_Uu7JwUd9e6Nk8o2_EUfxWqsun"
!gdown --id "1uSmhXzyV1Zvb4TJJCzpsZOIcw7CCJLxj"

In [ ]:
%%capture
# == Install dependencies ==
!pip3 install cython
!pip3 install 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
!pip3 install cython_bbox

%cd /content/ByteTrack/
!pip3 install -r requirements.txt

In [ ]:
%%capture
# == Install ByteTrack ==
!python3 setup.py develop

### Video pipeline

In [ ]:
!pip install loguru
!pip install dotmap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
import yaml
import time
import os.path as osp
from dotmap import DotMap

import cv2
import numpy as np
import torch
import torchvision

from loguru import logger

In [ ]:
# Read config file
config = None
with open(CONFIG_FILE, "r") as stream:
    config = yaml.safe_load(stream)
    config = DotMap(config)

In [ ]:
# Create save folder
current_time = time.localtime()
timestamp = time.strftime("%Y_%m_%d_%H_%M_%S", current_time)
save_folder = osp.join(TEMP_DIR, timestamp)
save_path = osp.join(save_folder, input_video_fn)

In [ ]:
# Read input video
cap = cv2.VideoCapture(INPUT_VIDEO)

# Get main video characteristic
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH)  # float
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float
fps = cap.get(cv2.CAP_PROP_FPS) # float

vid_writer = cv2.VideoWriter(
    save_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (int(width), int(height))
)

In [ ]:
save_path

'/content/tmp_video/2022_09_09_00_54_55/08fd33_0.mp4'

### Object detection model

In [ ]:
# Add path to model to syspath
PATH_TO_MODEL = "/content/ByteTrack/exps/example/mot"
PATH_TO_CHECKPOINT = "/content/ByteTrack/pretrained/bytetrack_m_mot17.pth.tar"

sys.path.append(PATH_TO_MODEL)

In [ ]:
from yolox_m_mix_det import Exp

exp = Exp()

if config.conf is not None:
    exp.test_conf = config.conf
if config.nms is not None:
    exp.nmsthre = config.nms
if config.tsize is not None:
    exp.test_size = (config.tsize, config.tsize)

In [ ]:
def preprocess(image, input_size, mean, std, swap=(2, 0, 1)):
  if len(image.shape) == 3:
      padded_img = np.ones((input_size[0], input_size[1], 3)) * 114.0
  else:
      padded_img = np.ones(input_size) * 114.0
  img = np.array(image)
  r = min(input_size[0] / img.shape[0], input_size[1] / img.shape[1])
  resized_img = cv2.resize(
      img,
      (int(img.shape[1] * r), int(img.shape[0] * r)),
      interpolation=cv2.INTER_LINEAR,
  ).astype(np.float32)
  padded_img[: int(img.shape[0] * r), : int(img.shape[1] * r)] = resized_img

  padded_img = padded_img[:, :, ::-1]
  padded_img /= 255.0
  if mean is not None:
      padded_img -= mean
  if std is not None:
      padded_img /= std
  padded_img = padded_img.transpose(swap)
  padded_img = np.ascontiguousarray(padded_img, dtype=np.float32)
  return padded_img, r

def postprocess(prediction, num_classes, conf_thre=0.7, nms_thre=0.45):
  box_corner = prediction.new(prediction.shape)
  box_corner[:, :, 0] = prediction[:, :, 0] - prediction[:, :, 2] / 2
  box_corner[:, :, 1] = prediction[:, :, 1] - prediction[:, :, 3] / 2
  box_corner[:, :, 2] = prediction[:, :, 0] + prediction[:, :, 2] / 2
  box_corner[:, :, 3] = prediction[:, :, 1] + prediction[:, :, 3] / 2
  prediction[:, :, :4] = box_corner[:, :, :4]

  output = [None for _ in range(len(prediction))]
  for i, image_pred in enumerate(prediction):

      # If none are remaining => process next image
      if not image_pred.size(0):
          continue
      # Get score and class with highest confidence
      class_conf, class_pred = torch.max(
          image_pred[:, 5 : 5 + num_classes], 1, keepdim=True
      )

      conf_mask = (image_pred[:, 4] * class_conf.squeeze() >= conf_thre).squeeze()
      # _, conf_mask = torch.topk((image_pred[:, 4] * class_conf.squeeze()), 1000)
      # Detections ordered as (x1, y1, x2, y2, obj_conf, class_conf, class_pred)
      detections = torch.cat((image_pred[:, :5], class_conf, class_pred.float()), 1)
      detections = detections[conf_mask]
      if not detections.size(0):
          continue

      nms_out_index = torchvision.ops.batched_nms(
          detections[:, :4],
          detections[:, 4] * detections[:, 5],
          detections[:, 6],
          nms_thre,
      )
      detections = detections[nms_out_index]
      if output[i] is None:
          output[i] = detections
      else:
          output[i] = torch.cat((output[i], detections))
  return output


In [ ]:
class ObjectDetection(object):
    def __init__(self, exp, weight_file, device):
        self.model = exp.get_model().to(device)
        self.num_classes = exp.num_classes
        self.confthre = exp.test_conf
        self.nmsthre = exp.nmsthre
        self.test_size = exp.test_size
        self.device = device
        
        self.rgb_means = (0.485, 0.456, 0.406)
        self.std = (0.229, 0.224, 0.225)

        self.model.eval()
        checkpoint = torch.load(weight_file, map_location="cpu")
        self.model.load_state_dict(checkpoint["model"])  

    def inference(self, img, timer):
        img_info = {}

        height, width = img.shape[:2]
        img_info["height"] = height
        img_info["width"] = width
        img_info["raw_img"] = img

        img, ratio = preprocess(img, self.test_size, self.rgb_means, self.std)

        img_info["ratio"] = ratio
        img = torch.from_numpy(img).unsqueeze(0).float().to(self.device)
        with torch.no_grad():
            timer.tic()
            outputs = self.model(img)
        
        outputs = postprocess(outputs, self.num_classes, self.confthre, self.nmsthre)
        
        #logger.info("Infer time: {:.4f}s".format(time.time() - t0))
        return outputs, img_info

In [ ]:
predictor = ObjectDetection(exp, PATH_TO_CHECKPOINT, config.device)

### Tracking model

In [ ]:
class ObjectTracking(object):
  def __init__(self, tracker, config):
    self.results = []
    self.tracker = tracker
    self.config = config

  def update(self, frime_id,output_results, img_info, img_size):
    online_targets = self.tracker.update(output_results, img_info, img_size)
    online_tlwhs = []
    online_ids = []
    online_scores = []
    for t in online_targets:
        tlwh = t.tlwh
        tid = t.track_id
        vertical = tlwh[2] / tlwh[3] > self.config.aspect_ratio_thresh
        if tlwh[2] * tlwh[3] > self.config.min_box_area and not vertical:
            online_tlwhs.append(tlwh)
            online_ids.append(tid)
            online_scores.append(t.score)
            self.results.append(
                f"{frame_id},{tid},{tlwh[0]:.2f},{tlwh[1]:.2f},{tlwh[2]:.2f},{tlwh[3]:.2f},{t.score:.2f},-1,-1,-1\n"
            )
    return online_tlwhs, online_ids, online_scores


In [ ]:
from yolox.tracker.byte_tracker import BYTETracker

tracker = ObjectTracking(BYTETracker(config, frame_rate=fps), config)

### Video processing

In [ ]:
from yolox.tracking_utils.timer import Timer
from yolox.utils.visualize import plot_tracking

In [ ]:
timer = Timer()
frame_id = 0

while True:
    if frame_id % 20 == 0:
        logger.info('Processing frame {} ({:.2f} fps)'.format(frame_id, 1. / max(1e-5, timer.average_time)))
    ret_val, frame = cap.read()
    if ret_val:
        outputs, img_info = predictor.inference(frame, timer)
        if outputs[0] is not None:
            online_tlwhs, online_ids, online_scores = tracker.update(outputs[0], [img_info['height'], img_info['width']], exp.test_size)
            timer.toc()
            online_im = plot_tracking(img_info['raw_img'], online_tlwhs, online_ids, frame_id=frame_id + 1, fps=1. / timer.average_time)
        else:
            timer.toc()
            online_im = img_info['raw_img']
        if config.save_result:
            vid_writer.write(online_im)
    else:
        break
    frame_id += 1

if config.save_result:
    res_file = osp.join(OUTPUT_DIR, f"{timestamp}.txt")
    with open(res_file, 'w') as f:
        f.writelines(tracker.results)
    logger.info(f"save results to {res_file}")

2022-09-09 00:57:11.396 | INFO     | __main__:<module>:6 - Processing frame 0 (100000.00 fps)
2022-09-09 00:57:15.167 | INFO     | __main__:<module>:6 - Processing frame 20 (15.66 fps)
2022-09-09 00:57:18.921 | INFO     | __main__:<module>:6 - Processing frame 40 (16.33 fps)
2022-09-09 00:57:22.422 | INFO     | __main__:<module>:6 - Processing frame 60 (17.22 fps)
2022-09-09 00:57:25.858 | INFO     | __main__:<module>:6 - Processing frame 80 (17.92 fps)
2022-09-09 00:57:29.253 | INFO     | __main__:<module>:6 - Processing frame 100 (18.47 fps)
2022-09-09 00:57:32.646 | INFO     | __main__:<module>:6 - Processing frame 120 (18.95 fps)
2022-09-09 00:57:35.960 | INFO     | __main__:<module>:6 - Processing frame 140 (19.38 fps)
2022-09-09 00:57:39.278 | INFO     | __main__:<module>:6 - Processing frame 160 (19.76 fps)
2022-09-09 00:57:42.546 | INFO     | __main__:<module>:6 - Processing frame 180 (20.04 fps)
2022-09-09 00:57:45.841 | INFO     | __main__:<module>:6 - Processing frame 200 (2

In [ ]:
# == Get rendered result video file path ==
import re
%cd /content/ByteTrack
with open('log.txt', 'r') as file:
    text = file.read().replace('\n', '')

m = re.search('video save_path is ./(.+?).mp4', text)
if m:
    found = '/content/ByteTrack/' + m.group(1) + ".mp4"
found

In [ ]:
import shutil
shutil.copy(found, TEMP_DIR)

NameError: ignored

In [ ]:
TMP_VIDEO = f"{TEMP_DIR}/{input_video_fn}"

In [ ]:
!cp /content/drive/MyDrive/tracking/1606b0e6_0.mp4 /content/tmp_video/1606b0e6_0.mp4

## Cut video by given timestamp 

In [ ]:
!pip install minio
!pip install imageio_ffmpeg

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.9 MB 1.4 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import uuid
import time
import pandas as pd
from tqdm import tqdm

import subprocess
from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip

from minio import Minio
from minio.error import S3Error

import psycopg2
from psycopg2 import OperationalError


In [ ]:
MARKUP_FILE = "/content/drive/MyDrive/kaggle/datasets/dfl-bundesliga-data-shootout/train.csv"
TIME_WINDOW = 5

### Minio Setting

In [ ]:
MINIO_ENDPOINT_URL = 
MINIO_ACCESS_KEY = 
MINIO_SECRET_KEY = 
MINIO_BUCKET = 

client = Minio(
        MINIO_ENDPOINT_URL,
        access_key=MINIO_ACCESS_KEY,
        secret_key=MINIO_SECRET_KEY,
        secure=False
)

def add_file_to_s3(input_path, object_name):
    found = client.bucket_exists(MINIO_BUCKET)
    assert found == True
    client.fput_object(MINIO_BUCKET, object_name, input_path)

def get_file_to_s3(object_name, file_path):
    found = client.bucket_exists(MINIO_BUCKET)
    assert found == True
    client.fget_object(MINIO_BUCKET, object_name, file_path)

### Postgres Setting

In [ ]:
POSTGRES_USER=
POSTGRES_PASSWORD=
POSTGRES_DB=
POSTGRES_HOST=
POSTGRES_PORT=

def pg_create_connection(db_name, db_user, db_password, db_host, db_port):
    connection = None
    try:
        connection = psycopg2.connect(database=db_name,
                                      user=db_user,
                                      password=db_password,
                                      host=db_host,
                                      port=db_port,)
        connection.autocommit = True
    except OperationalError as ex:
        print(f"The ERROR {ex} occurred")
    return connection

connection = pg_create_connection(POSTGRES_DB, 
                                  POSTGRES_USER, 
                                  POSTGRES_PASSWORD, 
                                  POSTGRES_HOST,
                                  POSTGRES_PORT)

def pg_insert_query(query, param=()):
    with connection.cursor() as cursor:
        try:
            cursor.execute(query, param)
        except OperationalError as ex:
            print(f"The ERROR {ex} occurred")

def pg_one_select_query(query, param=()):
    with connection.cursor() as cursor:
        try:
            cursor.execute(query, param)
            hander = cursor.fetchone()[0]
            return hander
        except OperationalError as ex:
            print(f"The ERROR {ex} occurred")

def pg_many_select_query(query, param=()):
    with connection.cursor() as cursor:
        try:
            cursor.execute(query, param)
            return cursor.fetchall()
        except OperationalError as ex:
          print(f"The ERROR {ex} occurred")

def add_clip_to_db(video_id, action_type, stime, clip_path):
  action_id = pg_one_select_query(query=f"""
                                  SELECT id FROM public.action_types
                                  WHERE name=%s;
                                  """, param=(action_type,))
  query = f"""
      INSERT INTO events(id_video, id_action, time, clip_path)
      VALUES (%s, %s, %s, %s)
      RETURNING id;
  """
  pg_insert_query(query=query,
                  param=(video_id, action_id, stime, clip_path))

### Get preview from video

In [ ]:
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
query_video_id = pg_many_select_query(query=f"""
                                SELECT id FROM public.videos
                                WHERE name=%s;
                                 """, param=(input_video_name,))
if len(query_video_id):
  video_id = query_video_id[0][0]
else:
  # Get preview
  cap = cv2.VideoCapture(INPUT_VIDEO)
  cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
  res, frame = cap.read()
  suuid = str(uuid.uuid4())
  filename_preview = f"{TEMP_DIR}/{str(uuid.uuid4())}.jpg"
  cv2.imwrite(filename_preview, frame)
  
  # Add to dp and s3
  add_file_to_s3(filename_preview, f'{input_video_name}/{suuid}.jpg')
  video_id = pg_one_select_query(query=f"""
                                INSERT INTO videos(name, preview_path) 
                                VALUES (%s)
                                RETURNING id;
                                """, param=(input_video_name, f'{input_video_name}/{suuid}.jpg'))
  # Remove tmp file
  os.remove(filename_preview)

### Main processing

In [ ]:
markup = pd.read_csv(MARKUP_FILE)
video_markup = markup[~(markup.event.isin(['start', 'end']))]
video_markup = video_markup[video_markup.video_id == input_video_name]
video_markup = video_markup[video_markup.time < 900]

In [ ]:
video_markup

,video_id,time,event,event_attributes
1,1606b0e6_0,201.15,challenge,['ball_action_forced']
4,1606b0e6_0,210.87,challenge,['opponent_dispossessed']
7,1606b0e6_0,219.23,throwin,['pass']
10,1606b0e6_0,224.43,play,"['pass', 'openplay']"
13,1606b0e6_0,229.39,play,"['pass', 'openplay']"
...,...,...,...,...
268,1606b0e6_0,878.03,play,"['pass', 'openplay']"
271,1606b0e6_0,884.55,play,"['pass', 'openplay']"
274,1606b0e6_0,889.03,play,"['pass', 'openplay']"
277,1606b0e6_0,893.35,play,"['pass', 'openplay']"


In [ ]:
for index, row in tqdm(video_markup.iterrows()):
  timestamp = row["time"]
  action = row["event"]
  # Calc time for subclip
  start_t = max(timestamp - TIME_WINDOW, 0)
  # TODO: Find video time duration 
  end_t = timestamp + TIME_WINDOW

  suuid = str(uuid.uuid4())
  filename_mp4 = f"{OUTPUT_DIR}/{suuid}.mp4"
  filename_h264 = f"{OUTPUT_DIR}/{suuid}.h264"

  # Extract subclip
  cut_command = f"ffmpeg -ss {start_t} -i {TMP_VIDEO} -t {2*TIME_WINDOW} -async 1 {filename_mp4}"
  process = subprocess.run(cut_command.split(), stdout=subprocess.PIPE)

  # Extract subclip
  cut_command = f"ffmpeg -i {filename_mp4} -an -vcodec libx264 -crf 23 {filename_h264}"
  process = subprocess.run(cut_command.split(), stdout=subprocess.PIPE)

  add_file_to_s3(filename_h264, f'{input_video_name}/{suuid}.h264')
  add_clip_to_db(video_id, action, timestamp, f'{input_video_name}/{suuid}.h264')

  os.remove(filename_mp4)
  os.remove(filename_h264)

59it [59:54, 59.13s/it]

In [ ]:
!rm -rf /content/clip_video/*